<a href="https://colab.research.google.com/github/SpineCrow/A-bundle-of-Graphical-User-Interface-projects/blob/main/Final_Source_Code_For_PneuVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below: Application in Gradio using ResNet18

In [ ]:
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
import cv2
import matplotlib.pyplot as plt
import gradio as gr

# -------------------- MODEL SETUP --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load(
    "/content/drive/MyDrive/ResNet18 Model and Testing Images/ResNet18model.pth",
    map_location=device
))
model.to(device)
model.eval()

class_names = ["NORMAL", "PNEUMONIA"]

#Uncertainty threshold to trigger flagging
UNCERTAINTY_THRESHOLD = 0.7
# -------------------- DCE FUNCTION --------------------
def DCE(pil_image):
    gray = np.array(pil_image.convert("L"))
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl1 = clahe.apply(gray)
    cl1_rgb = cv2.cvtColor(cl1, cv2.COLOR_GRAY2RGB)
    return Image.fromarray(cl1_rgb)

# -------------------- GRAD-CAM --------------------
def generate_heatmap(model, input_tensor, target_class):
    gradients = []
    activations = []

    def forward_hook(module, input, output):
        activations.append(output.detach())

    def backward_hook(module, grad_input, grad_output):
        gradients.append(grad_output[0].detach())

    target_layer = model.layer4[-1].conv2
    forward_handle = target_layer.register_forward_hook(forward_hook)
    backward_handle = target_layer.register_backward_hook(backward_hook)

    # Forward + backward
    output = model(input_tensor)
    model.zero_grad()
    class_score = output[0, target_class]
    class_score.backward()

    grads = gradients[0]
    acts = activations[0]
    weights = grads.mean(dim=(2, 3), keepdim=True)
    cam = (weights * acts).sum(dim=1, keepdim=True)
    cam = F.relu(cam)
    cam = cam.squeeze().cpu().numpy()
    cam = cv2.resize(cam, (224, 224))
    cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)

    forward_handle.remove()
    backward_handle.remove()
    return cam

def overlay_heatmap(pil_image, heatmap):
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    img = np.array(pil_image.convert("RGB"))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    overlay = cv2.addWeighted(heatmap, 0.4, img, 0.6, 0)
    return cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)

# -------------------- IDENTIFICATION FUNCTION --------------------
def IdentifyImage(pil_image, dynamicEnhance):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5])
    ])

    if dynamicEnhance:
        pil_image = DCE(pil_image)

    input_tensor = transform(pil_image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        probabilities = F.softmax(output, dim=1)[0]
        predicted = torch.argmax(probabilities).item()
        confidence = probabilities[predicted].item() * 100
    # Determining Uncertainty
    is_uncertain = confidence < UNCERTAINTY_THRESHOLD
    uncertainty_flag = " (Uncertain, needs review)" if is_uncertain else ""
    # Grad-CAM visualization
    heatmap = generate_heatmap(model, input_tensor, predicted)
    overlay = overlay_heatmap(pil_image.resize((224, 224)), heatmap)
    overlay_pil = Image.fromarray(overlay)

    result_text = f"Prediction: {class_names[predicted]} ({confidence:.2f}%) {uncertainty_flag}"
    return result_text, overlay_pil

# -------------------- GRADIO INTERFACE --------------------
with gr.Blocks() as demo:
    gr.Markdown("PneuVision Interface - Chest X-Ray Classifier with Grad-CAM using ResNet18")

    with gr.Row():
        img = gr.Image(type="pil", image_mode="L", label="Upload X-Ray")
        apply_dce_checkbox = gr.Checkbox(label="Apply Dynamic Contrast Enhancement (DCE)", value=True)

    output_text = gr.Textbox(label="Prediction")
    output_heatmap = gr.Image(label="Grad-CAM Heatmap")

    btn = gr.Button("Analyze")
    btn.click(IdentifyImage, inputs=[img, apply_dce_checkbox], outputs=[output_text, output_heatmap])

demo.launch(share=True)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aae17d1bfe645fac03.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Below: Application in Gradio using ResNet50

In [ ]:
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
import cv2
import matplotlib.pyplot as plt
import gradio as gr

# -------------------- MODEL SETUP --------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)
model.load_state_dict(torch.load(
    "/content/drive/MyDrive/ResNet18 Model and Testing Images/ResNet50model_prototype.pth",
    map_location=device
))
model.to(device)
model.eval()

class_names = ["NORMAL", "PNEUMONIA"]

#Uncertainty threshold to trigger flagging
UNCERTAINTY_THRESHOLD = 0.7
# -------------------- DCE FUNCTION --------------------
def DCE(pil_image):
    gray = np.array(pil_image.convert("L"))
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl1 = clahe.apply(gray)
    cl1_rgb = cv2.cvtColor(cl1, cv2.COLOR_GRAY2RGB)
    return Image.fromarray(cl1_rgb)

# -------------------- GRAD-CAM --------------------
def generate_heatmap(model, input_tensor, target_class):
    gradients = []
    activations = []

    def forward_hook(module, input, output):
        activations.append(output.detach())

    def backward_hook(module, grad_input, grad_output):
        gradients.append(grad_output[0].detach())

    target_layer = model.layer4[-1].conv2
    forward_handle = target_layer.register_forward_hook(forward_hook)
    backward_handle = target_layer.register_backward_hook(backward_hook)

    # Forward + backward
    output = model(input_tensor)
    model.zero_grad()
    class_score = output[0, target_class]
    class_score.backward()

    grads = gradients[0]
    acts = activations[0]
    weights = grads.mean(dim=(2, 3), keepdim=True)
    cam = (weights * acts).sum(dim=1, keepdim=True)
    cam = F.relu(cam)
    cam = cam.squeeze().cpu().numpy()
    cam = cv2.resize(cam, (224, 224))
    cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)

    forward_handle.remove()
    backward_handle.remove()
    return cam

def overlay_heatmap(pil_image, heatmap):
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    img = np.array(pil_image.convert("RGB"))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    overlay = cv2.addWeighted(heatmap, 0.4, img, 0.6, 0)
    return cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)

# -------------------- IDENTIFICATION FUNCTION --------------------
def IdentifyImage(pil_image, dynamicEnhance):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5])
    ])

    if dynamicEnhance:
        pil_image = DCE(pil_image)

    input_tensor = transform(pil_image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        probabilities = F.softmax(output, dim=1)[0]
        predicted = torch.argmax(probabilities).item()
        confidence = probabilities[predicted].item() * 100
    # Determining Uncertainty
    is_uncertain = confidence < UNCERTAINTY_THRESHOLD
    uncertainty_flag = " (Uncertain, needs review)" if is_uncertain else ""
    # Grad-CAM visualization
    heatmap = generate_heatmap(model, input_tensor, predicted)
    overlay = overlay_heatmap(pil_image.resize((224, 224)), heatmap)
    overlay_pil = Image.fromarray(overlay)

    result_text = f"Prediction: {class_names[predicted]} ({confidence:.2f}%) {uncertainty_flag}"
    return result_text, overlay_pil

# -------------------- GRADIO INTERFACE --------------------
with gr.Blocks() as demo:
    gr.Markdown("PneuVision Interface - Chest X-Ray Classifier with Grad-CAM using ResNet50")

    with gr.Row():
        img = gr.Image(type="pil", image_mode="L", label="Upload X-Ray")
        apply_dce_checkbox = gr.Checkbox(label="Apply Dynamic Contrast Enhancement (DCE)", value=True)

    output_text = gr.Textbox(label="Prediction")
    output_heatmap = gr.Image(label="Grad-CAM Heatmap")

    btn = gr.Button("Analyze")
    btn.click(IdentifyImage, inputs=[img, apply_dce_checkbox], outputs=[output_text, output_heatmap])

demo.launch(share=True)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6d9949e53e1688b8ef.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Below: Training Block used for initializing and saving models for future use

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms # Import transforms
import torchvision.datasets as datasets # Import datasets
import torchvision.models as models # Import models
import numpy as np
import os

def UpdateTransform():
    return transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    # Added a random vertical flip to the dataset
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

transform = UpdateTransform()
fixed_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

# Load dataset folders
train_dataset = datasets.ImageFolder(root="/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/train", transform=transform)
val_dataset = datasets.ImageFolder(root="/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val", transform=fixed_transform)
test_dataset = datasets.ImageFolder(root="/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/test", transform=fixed_transform)

# Create loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

#Load neural network
#model = models.resnet18(pretrained=True)
#model = models.resnet50(pretrained=True) #2nd Model
model = models.densenet121(pretrained=True) #3rd Model
#Next is DenseNet-121
#model.fc = nn.Linear(model.fc.in_features, 2)

#Moves workload to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(device)

#Accuracy criteria
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

def TrainModel():
  #Trains on entire dataset for x different iterations
  num_epochs = 25
  #Training loop
  for epoch in range(num_epochs):
      #Sets model to train, running_loss keeps track of accuracy
      model.train()
      running_loss = 0.0
      iteration_count = 0

      if (epoch + 1) % 10 == 0:
            transform = UpdateTransform()
            train_dataset.transform = transform

      for images, labels in train_loader:
          images = images.to(device)
          labels = labels.to(device)

          optimizer.zero_grad()
          outputs = model(images)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item()
          iteration_count += 1
      print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")
      ValidateModel(val_loader)

def ValidateModel(loader_set):
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
      for images, labels in loader_set:
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print(f'Validation Accuracy: {100 * correct / total:.2f}%')


if __name__ == "__main__":
  ValidateModel(test_loader)
  TrainModel()
  ValidateModel(test_loader)
  #torch.save(model.state_dict(), "ResNet18model_prototype.pth")
  #torch.save(model.state_dict(), "ResNet50model_prototype.pth")
  torch.save(model.state_dict(), "DenseNet121model_prototype.pth")
  print("Model saved")
